In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/AlexeyAB/darknet/

Cloning into 'darknet'...
remote: Enumerating objects: 15283, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 15283 (delta 39), reused 59 (delta 25), pack-reused 15186
Receiving objects: 100% (15283/15283), 13.78 MiB | 12.77 MiB/s, done.
Resolving deltas: 100% (10350/10350), done.


In [3]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

In [ ]:
object_data = '/content/drive/MyDrive/Projects/BCCD/phase1/WBC-obj.data'
yolov3_cfg = '/content/drive/MyDrive/Projects/BCCD/phase1/yolo_custom.cfg'
preweights = '/content/darknet/darknet53.conv.74'
!./darknet detector train '$object_data' '$yolov3_cfg' '$preweights' -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 13302, rewritten_bbox = 0.466095 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.944164), count: 1, class_loss = 0.003746, iou_loss = 0.034072, total_loss = 0.037819 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000002, iou_loss = 0.000000, total_loss = 0.000002 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 13303, rewritten_bbox = 0.466060 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.566808, iou_loss = 0.000000, total_loss = 0.566808 
v3 (mse

Testing

In [6]:
import numpy as np
import pandas as pd
import keras
from keras import applications
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Bidirectional, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization, Input, concatenate, Reshape, LSTM
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import xml.etree.ElementTree as ET
import sklearn
import itertools
import cv2
import scipy
import os
import csv
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

In [7]:
folder='/content/drive/My Drive/Bloodcells/TRAIN'
count=0
train_file=open("data/train.txt","w")
for image in os.listdir(folder):
  x='/content/drive/My Drive/Bloodcells/TRAIN/'+image
  for img in os.listdir(x):
    image1=x+'/'+img
    if(image1[len(image1)-3:]!="txt"):
      train_file.write(image1)
      train_file.write('\n')
      print(image1)
     #count=count+1

Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_157_8281.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_102_1698.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_101_823.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_0_5763.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_100_8776.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_100_5599.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_101_8990.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_0_7337.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_100_8557.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_102_2561.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_100_8981.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_100_3946.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_0_4892.jpeg
/content/drive/My Drive/Bloodcells/TRAIN/NEUTROPHIL/_101_7628.jpeg
/content/drive/My 

In [33]:
!make

chmod +x *.sh
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/detector.c -o obj/detector.o
./src/detector.c: In function ‘train_detector’:
./src/detector.c:386:72: warning: suggest parentheses around ‘&&’ within ‘||’ [-Wparentheses]
             (iteration >= (iter_save + 1000) || iteration % 1000 == 0) && net.max_batches < 10000)
             ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~
./src/detector.c: In function ‘print_cocos’:
./src/detector.c:486:29: warning: format not a string literal and no format arguments [-Wformat-security]
                 fprintf(fp, buff);
                             ^~~~
./src/detector.c: In function ‘eliminate_bdd’:
./src/detector.c:579:21: warning: statement with no ef

In [36]:
object_data = '/content/drive/MyDrive/Projects/BCCD/phase1/WBC-obj.data'
yolov3_cfg = '/content/drive/MyDrive/Projects/BCCD/phase1/yolo_custom.cfg'
sw='/content/drive/MyDrive/phase1_weights/yolo_custom_last.weights'
phase2_train_data= '/content/drive/MyDrive/Phase2/phase2_train_data.txt'

In [ ]:
!./darknet detector test '$object_data' '$yolov3_cfg' '$sw' -thresh 0.70 -dont_show -save_labels < data/train.txt> '$phase2_train_data'

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   3 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   5 conv    128       3 x 3/ 2    304 x 304 x  64 ->  152 x 152 x 128 3.407 BF
   6 conv     64       1 x 1/ 1    152 x 152 x 128 ->  152 x 152 x  64 0.379 BF
   7 conv    128       3 x 3/ 1    152 x 152 x  64 ->  152 x 152 x 128 3.407 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 152 x 152 x 128 0.003 BF
   9 conv     64       1 x 1/ 1    152 x 152 x 128 ->  152 x 152 x  64 0.379 